In [39]:
from TSPGraph import Graph
import numpy as np
from numba import jit
from numba import njit, prange
import copy
from tqdm import tqdm
from multiprocessing import Pool
import time
from functools import lru_cache

In [14]:
graph = Graph('C')
obj = graph.objective
dist = graph.dist_matrix

/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [15]:
@jit
def random_solution():
    sol = np.random.choice(np.arange(200),size=100,replace=False)
    return np.append(sol,sol[0])

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/3766937091.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def random_solution():


In [16]:
combination_pairs = [(x,y) for x in range(1,100) for y in range(x+2, 100)]
combination_pairs_edges = [(x,y) for x in range(1,100) for y in range(x+2, 100)]

In [17]:
@jit
def ExchangeVertices(solution, i, j):
    result = copy.deepcopy(solution)
    result[[i,j]] = result[[j,i]]
    return result
@jit
def ExchangeVerticesDelta(dist, solution, i, j):
    res = (dist[solution[i-1], solution[j]] + dist[solution[j], solution[i+1]] + dist[solution[j-1], solution[i]] + dist[solution[i], solution[j+1]]) - (dist[solution[i-1], solution[i]] + dist[solution[i], solution[i+1]] + dist[solution[j-1], solution[j]] + dist[solution[j], solution[j+1]])
    return res
@jit
def MinInAllExchangedVertices(dist, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(dist, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return [minDelta, resultExchange]
@jit
def GreedyExchangeVertex(dist, solution):
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(dist, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None


MinInAllExchangedVertices(dist, random_solution())

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/2949073171.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeVertices(solution, i, j):
/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/2949073171.py:7: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeVerticesDelta(dist, solution, i, j):
/var/folders/vt/8kg_3

[-9611, (40, 95)]

In [18]:
@jit
def ExchangeEdges(solution, i, j):
    assert i<j
    result = copy.deepcopy(solution)
    result = np.concatenate((result[:i], result[j:i-1:-1], result[j+1:]))
    return result
@jit
def ExchangeEdgesDelta(dist, solution, i, j):
    assert i<j
    res = (dist[solution[i-1], solution[j]] + dist[solution[i], solution[j+1]]) - (dist[solution[i-1], solution[i]] + dist[solution[j], solution[j+1]])
    return res
@jit
def MinInAllExchangedEdges(dist, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(dist, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return (minDelta, resultExchange)
@jit
def GreedyExchangeEdges(dist, solution):
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(dist, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None

MinInAllExchangedEdges(dist, random_solution())

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/432162731.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeEdges(solution, i, j):
/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/432162731.py:8: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeEdgesDelta(dist, solution, i, j):
/var/folders/vt/8kg_3c6554l1g

(-6253, (24, 50))

In [19]:
@jit
def SwapVertexWithNewOne(solution, i, a):
    result = copy.deepcopy(solution)
    result[i] = a
    return result
@jit
def SwapVertexWithNewOneDelta(obj, solution, i, a):
    return (obj[solution[i-1], a] + obj[a, solution[i+1]]) - (obj[solution[i-1], solution[i]] + obj[solution[i], solution[i+1]])
@jit
def MinInAllSwaps(solution, obj):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    minDelta = 0
    resultSwap = tuple()
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(obj, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultSwap = (x[0], x[1])
    return (minDelta, resultSwap)
@jit
def GreedySwap(solution, obj):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(obj, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None
print(MinInAllSwaps(random_solution(), obj))

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/432162731.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "ExchangeEdges" failed type inference due to: Unknown attribute 'deepcopy' of type Module(<module 'copy' from '/opt/homebrew/Cellar/python@3.10/3.10.7/Frameworks/Python.framework/Versions/3.10/lib/python3.10/copy.py'>)

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/432162731.py", line 4:
<source missing, REPL/exec in use?>

During: typing of get attribute at /var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/432162731.py (4)

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/432162731.py", line 4:
<source missing, REPL/exec in use?>

  @jit
/opt/homebrew/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "ExchangeEdges" was compiled in object mode without forceobj=True.

File "

(-5300, (27, 169))


/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/3288179290.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "SwapVertexWithNewOne" failed type inference due to: Unknown attribute 'deepcopy' of type Module(<module 'copy' from '/opt/homebrew/Cellar/python@3.10/3.10.7/Frameworks/Python.framework/Versions/3.10/lib/python3.10/copy.py'>)

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/3288179290.py", line 3:
<source missing, REPL/exec in use?>

During: typing of get attribute at /var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/3288179290.py (3)

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/3288179290.py", line 3:
<source missing, REPL/exec in use?>

  @jit


In [20]:
@jit
def SteepestLocalSearch(obj, dist, initial_solution, type):
    current_solution = initial_solution
    max_iter = 1_000_000
    while(max_iter > 0):
        # print(current_cost)
        best_swap_val, best_swap = MinInAllSwaps(current_solution, obj)
        if type == 'edge':
            best_exchange_val, best_exchange = MinInAllExchangedEdges(dist, current_solution)
        else:
            best_exchange_val, best_exchange = MinInAllExchangedVertices(dist, current_solution)
        
        if best_swap_val >= 0 and best_exchange_val >= 0:
            break

        if best_swap_val < best_exchange_val:
            current_solution = SwapVertexWithNewOne(current_solution, best_swap[0], best_swap[1])
            # print('swap' + str(best_swap_val))
        elif type == 'edge':
            current_solution = ExchangeEdges(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))
        else:
            current_solution = ExchangeVertices(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))

        max_iter -= 1
    return current_solution
        
SteepestLocalSearch(obj, dist, random_solution(), 'edge')

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/2515397583.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def SteepestLocalSearch(obj, dist, initial_solution, type):
/opt/homebrew/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "SwapVertexWithNewOne" was compiled in object mode without forceobj=True.

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/3288179290.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/opt/homebrew/lib/python3.10/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back fro

array([178, 164,  34,  40, 118, 128, 132,  37, 143, 159,  59,  96, 147,
        27, 116,  46, 185, 181,  25,  64,  20,  71, 183,  61, 113, 163,
        74,  62, 155, 195,  36, 136,  54,  55,  22, 117,  53,  15, 108,
       171,  81, 154, 141,   6,  66,  98, 190,  72, 112,   5,  51, 135,
       101, 167,  99, 139, 110, 169,  95, 191,  31,  73,  94,  89, 111,
       197,  14,  80, 123, 124,   8, 125,  26, 119, 130,  92,  48, 152,
       189,  75, 177, 199,  77,   4, 114, 153,  88, 186, 194,  79,  21,
        76,   0, 149,  91,  50, 121,  43,  35,  19, 178])

In [40]:
@lru_cache(maxsize=1_000_000)
def addition_cost(graph, triplet):
    return graph.objective[triplet[0], triplet[1]] + graph.objective[triplet[1], triplet[2]] - graph.objective[triplet[0], triplet[2]]

In [123]:
@jit
def getRegret2Weighted(graph, cycle):
    candidates = np.delete(np.arange(200), cycle)
    costs = [-9999999] * 200
    solutions = [0] * 200
    weigth = 0.61
    scores = [-9999999] * 200
    temporaryCost = graph.cycle_cost(cycle)
    for candidate in candidates:
        currentCosts = []
        currentSolutions = []
        for i in range(len(cycle) - 1):
            currentCosts.append(graph.cycle_cost(cycle) + addition_cost(graph,(cycle[i],candidate,cycle[i+1])))
            currentSolutions.append(np.concatenate((cycle[:i + 1],np.array([candidate]).reshape((1,)), cycle[i + 1:])))
        firstBestId = np.argmin(currentCosts)
        firstBestCost = currentCosts[firstBestId]
        currentCosts = np.delete(currentCosts, firstBestId)
        secondBestId = np.argmin(currentCosts)
        secondBestCost = currentCosts[secondBestId]
        bestSolution = currentSolutions[firstBestId]

        newTemporaryCost = graph.cycle_cost(bestSolution)
        difference = newTemporaryCost - temporaryCost
        scores[candidate] = weigth * (secondBestCost - firstBestCost) - (1 - weigth) * difference
        costs[candidate] = newTemporaryCost
        solutions[candidate] = bestSolution

    bestRegretId = np.argmax(scores)
    cycle = solutions[bestRegretId]
    temporaryCost = costs[bestRegretId]

    return cycle

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/256898679.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def getRegret2Weighted(graph, cycle):


In [133]:
@jit
def destroyCycle(solution):
  numberOfNodesToDestroy = np.random.randint(5, 36)
  nodesToDestroy = [np.random.randint(1, 101) for i in range(numberOfNodesToDestroy)]

  for i in nodesToDestroy:
    solution[i] = -1

  return solution

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/1079767723.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def destroyCycle(solution):


In [125]:
@jit
def repairCycle(graph, cycle):
  cycle = [node for node in cycle if node != -1]

  while(len(cycle) < 101):
    cycle = getRegret2Weighted(graph, cycle)

  return cycle

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/1834971898.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def repairCycle(graph, cycle):


In [100]:
def LNS(obj, dist, timeLimit, graph):
  bestSolution = SteepestLocalSearch(obj, dist, random_solution(), 'edge')
  timeout = time.time() + timeLimit

  while(time.time() < timeout):
    newSolution = destroyCycle(bestSolution)
    newSolution = repairCycle(graph, newSolution)

    if graph.cycle_cost(newSolution) < graph.cycle_cost(bestSolution):
      bestSolution = newSolution

  return bestSolution, graph.cycle_cost(bestSolution)

In [117]:
def LNSExperiment(obj, dist, graph):
  solutions = []
  costs = []
  for i in range(20):
    bestSolution, bestCost = LNS(obj, dist, 10, graph)
    solutions.append(bestSolution)
    costs.append(bestCost)
  return solutions, costs

In [108]:
def LNSWithLocalSearch(obj, dist, timeLimit, graph):
  bestSolution = SteepestLocalSearch(obj, dist, random_solution(), 'edge')
  timeout = time.time() + timeLimit

  while(time.time() < timeout):
    newSolution = destroyCycle(bestSolution)
    newSolution = repairCycle(graph, newSolution)
    newSolution = SteepestLocalSearch(obj, dist, newSolution, 'edge')

    if graph.cycle_cost(newSolution) < graph.cycle_cost(bestSolution):
      bestSolution = newSolution

  return bestSolution, graph.cycle_cost(bestSolution)

In [118]:
def LNSExperimentWithLocalSearch(obj, dist, graph):
  solutions = []
  costs = []
  for i in range(20):
    bestSolution, bestCost = LNSWithLocalSearch(obj, dist, 140, graph)
    solutions.append(bestSolution)
    costs.append(bestCost)
  return solutions, costs

In [119]:
graph = Graph('C')

solutionsC, costsC = LNSExperiment(graph.objective, graph.dist_matrix, graph)

/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [141]:
print(np.min(costsC), np.max(costsC), np.mean(costsC), np.median(costsC))

48464 50799 49496.85 49458.5


In [121]:
graph = Graph('C')

solutionsLocalSearchC, costsLocalSearchC = LNSExperimentWithLocalSearch(graph.objective, graph.dist_matrix, graph)

/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [140]:
print(np.min(costsLocalSearchC), np.max(costsLocalSearchC), np.mean(costsLocalSearchC), np.median(costsLocalSearchC))

47587 50257 49179.1 49256.0


In [128]:
graph = Graph('D')

solutionsD, costsD = LNSExperiment(graph.objective, graph.dist_matrix, graph)

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/1834971898.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "repairCycle" failed type inference due to: non-precise type pyobject
During: typing of argument at /var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/1834971898.py (1)

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/1834971898.py", line 1:
<source missing, REPL/exec in use?>

  @jit
/opt/homebrew/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "repairCycle" was compiled in object mode without forceobj=True.

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_35307/1834971898.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/opt/homebrew/lib/python3.10/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-b

In [137]:
print(np.min(costsD), np.max(costsD), np.mean(costsD), np.median(costsD))

44290 47145 45929.9 45885.5


In [139]:
graph = Graph('D')

solutionsLocalSearchD, costsLocalSearchD = LNSExperimentWithLocalSearch(graph.objective, graph.dist_matrix, graph)

KeyboardInterrupt: 

In [138]:
print(np.min(costsLocalSearchD), np.max(costsLocalSearchD), np.mean(costsLocalSearchD), np.median(costsLocalSearchD))

44235 48872 46221.1 46250.5
